In [2]:
import tensorflow as tf
import numpy as np
import gym

/Users/f.seiler/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
env = gym.make("CartPole-v0")

In [4]:
n_steps = 250
n_actions = 2
state_size = 4
entropy_coef = 0
vf_coef = 0.1
learning_rate = 0.003
lam = 0.95
gamma = 0.99
episodes = 2000

# A2C Network Architecture

In [5]:
# Input
X = tf.placeholder(tf.float32, shape=(None,state_size), name="X")
y = tf.placeholder(tf.int32, shape=(None,n_actions), name="y")
advantages = tf.placeholder(tf.float32, [None], name="advantages")
rewards = tf.placeholder(tf.float32, [None], name="rewards")

# Hidden
fc1 = tf.layers.dense(X, 32, activation=tf.nn.relu, kernel_initializer=
tf.contrib.layers.xavier_initializer())
fc2 = tf.layers.dense(fc1, 64, activation=tf.nn.relu, kernel_initializer=
tf.contrib.layers.xavier_initializer())
fc3 = tf.layers.dense(fc1, 10, activation=tf.nn.relu, kernel_initializer=
tf.contrib.layers.xavier_initializer())
fc4 = tf.layers.dense(fc3, n_actions, activation=None)
# Output
action_distribution = tf.nn.softmax(fc4)
value = tf.layers.dense(fc3, 1, activation=None, kernel_initializer=
tf.contrib.layers.xavier_initializer())

# Loss
neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc4, labels=y)
pg_loss = tf.reduce_mean(advantages * neg_log_prob)
vf_loss = tf.losses.mean_squared_error(tf.squeeze(value), rewards)

entropy = 0 # TODO: Implement entropy
loss = pg_loss - entropy * entropy_coef + vf_loss * vf_coef

# Gradient Clipping
params = tf.trainable_variables()

grads = tf.gradients(loss, params)
grads, grad_norm = tf.clip_by_global_norm(grads, 0.5)
grads = list(zip(grads, params))

# Training
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = trainer.apply_gradients(grads)

Instructions for updating:
Use the retry module or similar alternatives.


In [6]:
writer = tf.summary.FileWriter("./tensorboard/13", )

In [7]:
reward_mean = tf.placeholder(tf.float32, name="reward_mean")
policy_loss = tf.placeholder(tf.float32, name="policy_loss")
value_loss = tf.placeholder(tf.float32, name="value_loss")
tf.summary.scalar("Reward_mean", reward_mean)
tf.summary.scalar("Policy_loss", policy_loss)
tf.summary.scalar("Value_loss", value_loss)
write_op = tf.summary.merge_all()

In [8]:
def discount_with_dones(rewards, dones, gamma):
    discounted = []
    r = 0
    for reward, done in zip(rewards[::-1], dones[::-1]):
        r = reward + gamma*r*(1.-done) # fixed off by one bug
        discounted.append(r)
    return discounted[::-1]

In [ ]:

rewardlist = []
with tf.Session() as sess:
    reward, done = None, False
    states = env.reset()
    sess.run(tf.global_variables_initializer())
    i=0
    for ep in range(episodes):
        states = env.reset()
        rewardlist = []
        reward, done = None, False
        mb_states, mb_actions, mb_rewards, mb_values, mb_dones = [],[],[],[],[]
        
        
        render = False
        while not len(mb_dones) > n_steps:
            #if(render):
                #env.render()
            ad, values = sess.run([action_distribution, value], {X: states.reshape((1,state_size))})
            action = np.random.choice(range(ad.shape[1]), p=ad.ravel())  
            mb_states.append(states)
            _action = np.zeros(ad.shape[1])
            _action[action] = 1
            mb_actions.append(_action)
            mb_values.append(values)
            mb_dones.append([done])
    
            if(done):
                i+=1
                states = env.reset()
                reward = -2
                done = False
                rewardlist=[]
                if(i % 10 == 0):
                    render = True
                else:
                    render = False
            else:  
                states, reward, done, _ = env.step(action)
                
            rewardlist.append(reward)
            mb_rewards.append([reward])
        
        mb_rewards = np.asarray(mb_rewards, dtype=np.float32)
        #mb_rewards -= np.mean(mb_rewards)
        #mb_rewards /= np.std(mb_rewards)
        #print(mb_rewards)
        mb_actions = np.asarray(mb_actions)
        mb_values = np.asarray(mb_values, dtype=np.float32)
        mb_values = mb_values.flatten()
        mb_dones = np.asarray(mb_dones, dtype=np.bool)
        last_value = sess.run(value, {X: states.reshape((1,state_size))})

        '''
        lastgaelam = 0
        for t in reversed(range(len(mb_dones))):
            if t == len(mb_dones) - 1:
                nextnonterminal = 1.0 - done
                nextvalue = last_value
            else:
                nextnonterminal = 1.0 - mb_dones[t+1]
                nextvalue = mb_values[t+1]
            delta = mb_rewards[t] + gamma * nextvalue * nextnonterminal - mb_values[t]
            mb_advantages[t] = lastgaelam = delta + gamma * lam * nextnonterminal * lastgaelam
        #mb_values = np.asarray(mb_values, dtype=np.float32)
        mb_advantages = np.asarray(mb_advantages, dtype=np.float32)
        # Here we calculate advantage A(s,a) = R + yV(s') - V(s)
        # rewards = R + yV(s')
        advs = rewards - values
        '''
        mb_returns = np.zeros_like(mb_rewards)
        mb_advs = np.zeros_like(mb_rewards)
        lastgaelam = 0
        for t in reversed(range(len(mb_rewards))):
            if t == len(mb_rewards)  - 1:
                nextnonterminal = 1.0 - done
                nextvalues = last_value
            else:
                nextnonterminal = 1.0 - mb_dones[t+1]
                nextvalues = mb_values[t+1]
            delta = mb_rewards[t] + gamma * nextvalues * nextnonterminal - mb_values[t]
            mb_advs[t] = lastgaelam = delta + gamma * nextnonterminal * lastgaelam

        mb_advs = mb_advs.flatten()
        mb_returns = mb_advs + mb_values
        #mb_returns -= np.mean(mb_returns)
        #mb_returns /= np.std(mb_returns)
        #mb_advs -= np.mean(mb_advs)
        #mb_advs /= np.std(mb_advs)
        #print(mb_advantages)
        feed_dict = {X: mb_states,
                    y: mb_actions,
                    advantages: mb_advs, # Use to calculate our policy loss
                    rewards: mb_returns}

        policy_loss_, value_loss_, loss_, _ = sess.run([pg_loss, vf_loss, loss, train], feed_dict)
        print("Value:",value_loss_)
        print("Policy:",policy_loss_)
        print("Loss:",loss_)
        if(len(rewardlist)>0):
            mr = np.sum(rewardlist)
            #print(mr)
            summary = sess.run(write_op, feed_dict={reward_mean: mr, policy_loss: policy_loss_, value_loss: value_loss_})
            writer.add_summary(summary, ep)
            writer.flush()
    while True:
        state = env.reset()
        done = False
        while not done:
            env.render()
            ad, values = sess.run([action_distribution, value], {X: states.reshape((1,state_size))})
            action = np.random.choice(range(ad.shape[1]), p=ad.ravel())  
            states, reward, done, _ = env.step(action)

Value: 165.16132
Policy: 7.4956985
Loss: 24.011831
Value: 266.4716
Policy: 9.195281
Loss: 35.84244
Value: 164.59033
Policy: 7.158986
Loss: 23.61802
Value: 291.3391
Policy: 9.467096
Loss: 38.601006
Value: 508.56555
Policy: 11.51012
Loss: 62.366676
Value: 234.50237
Policy: 8.495623
Loss: 31.94586
Value: 276.55167
Policy: 9.025299
Loss: 36.680466
Value: 304.00082
Policy: 10.013335
Loss: 40.413418
Value: 124.77244
Policy: 6.1670227
Loss: 18.644268
Value: 536.59265
Policy: 12.598091
Loss: 66.257355
Value: 434.80963
Policy: 11.040947
Loss: 54.52191
Value: 193.18832
Policy: 7.463793
Loss: 26.782625
Value: 184.01843
Policy: 7.642622
Loss: 26.044466
Value: 255.70355
Policy: 9.309618
Loss: 34.879974
Value: 165.87569
Policy: 7.004975
Loss: 23.592543
Value: 167.15904
Policy: 7.231196
Loss: 23.9471
Value: 214.68343
Policy: 8.139285
Loss: 29.607628
Value: 287.07593
Policy: 8.970759
Loss: 37.678352
Value: 222.60558
Policy: 8.487042
Loss: 30.7476
Value: 352.39938
Policy: 10.448123
Loss: 45.688065
Valu

Value: 406.9934
Policy: -0.30679604
Loss: 40.392544
Value: 239.36066
Policy: -5.6558595
Loss: 18.280207
Value: 251.94743
Policy: -2.3544974
Loss: 22.840246
Value: 323.5787
Policy: -2.8041546
Loss: 29.553717
Value: 216.3686
Policy: -0.017694779
Loss: 21.619167
Value: 440.89954
Policy: 3.2802002
Loss: 47.370155
Value: 544.2345
Policy: 6.9358234
Loss: 61.359276
Value: 296.78363
Policy: -0.26944348
Loss: 29.40892
Value: 396.0478
Policy: 0.27923667
Loss: 39.884014
Value: 253.11829
Policy: -1.0257614
Loss: 24.286068
Value: 408.42505
Policy: 5.069028
Loss: 45.911533
Value: 310.7449
Policy: -3.7716844
Loss: 27.302807
Value: 306.02582
Policy: 0.3804629
Loss: 30.983046
Value: 257.39282
Policy: -0.26775426
Loss: 25.471529
Value: 545.78784
Policy: 5.882612
Loss: 60.461395
Value: 429.1307
Policy: 5.1134043
Loss: 48.026474
Value: 516.03723
Policy: 3.193643
Loss: 54.797367
Value: 270.3125
Policy: -0.45920238
Loss: 26.572048
Value: 382.85095
Policy: 0.14441451
Loss: 38.42951
Value: 768.931
Policy: 6.8

Value: 488.0419
Policy: 3.1348004
Loss: 51.93899
Value: 306.9183
Policy: 6.291862
Loss: 36.983692
Value: 527.67554
Policy: 3.9134443
Loss: 56.681
Value: 145.3454
Policy: -0.74114656
Loss: 13.793394
Value: 319.15598
Policy: 3.2454505
Loss: 35.16105
Value: 146.00174
Policy: 3.4847682
Loss: 18.084942
Value: 256.14932
Policy: 4.398672
Loss: 30.013605
Value: 265.2068
Policy: 4.410617
Loss: 30.931297
Value: 269.95404
Policy: -2.3455312
Loss: 24.649874
Value: 404.18408
Policy: 2.439906
Loss: 42.858315
Value: 140.97409
Policy: 1.8158534
Loss: 15.913262
Value: 378.8502
Policy: 2.353133
Loss: 40.238155
Value: 380.07428
Policy: 1.642223
Loss: 39.64965
Value: 410.5227
Policy: 0.50593334
Loss: 41.558205
Value: 232.62596
Policy: -4.5939865
Loss: 18.66861
Value: 921.0617
Policy: -14.726144
Loss: 77.38003
Value: 210.60524
Policy: -3.8132353
Loss: 17.247288
Value: 249.31985
Policy: -6.0013103
Loss: 18.930676
Value: 680.24817
Policy: -11.816833
Loss: 56.207985
Value: 230.80469
Policy: -4.578009
Loss: 18

Value: 72.68856
Policy: 1.5418286
Loss: 8.810684
Value: 479.96024
Policy: -0.08858257
Loss: 47.907444
Value: 148.3676
Policy: 2.0038812
Loss: 16.840641
Value: 221.24622
Policy: 4.3998113
Loss: 26.524433
Value: 319.31622
Policy: 0.80551755
Loss: 32.73714
Value: 70.437195
Policy: 2.203484
Loss: 9.247204
Value: 431.63715
Policy: 2.1617143
Loss: 45.325428
Value: 294.87653
Policy: -0.82379085
Loss: 28.663862
Value: 570.4477
Policy: 9.623182
Loss: 66.66795
Value: 125.14095
Policy: 3.754919
Loss: 16.269014
Value: 52.80225
Policy: 0.5772463
Loss: 5.8574715
Value: 375.21353
Policy: 1.3198549
Loss: 38.84121
Value: 405.83902
Policy: -0.56446373
Loss: 40.01944
Value: 160.1979
Policy: 0.029243592
Loss: 16.049034
Value: 225.5515
Policy: 6.602652
Loss: 29.157803
Value: 143.80458
Policy: 4.2227616
Loss: 18.60322
Value: 137.23357
Policy: -4.4861445
Loss: 9.237213
Value: 97.5758
Policy: 2.5804384
Loss: 12.338018
Value: 436.2876
Policy: -5.0674787
Loss: 38.561283
Value: 137.40598
Policy: 3.15802
Loss: 16

Value: 519.9388
Policy: -4.3394437
Loss: 47.654434
Value: 528.94025
Policy: -3.6949666
Loss: 49.19906
Value: 320.2202
Policy: -1.6178974
Loss: 30.404125
Value: 489.2866
Policy: -4.049772
Loss: 44.87889
Value: 112.79588
Policy: 4.7872996
Loss: 16.066889
Value: 347.46933
Policy: -2.229804
Loss: 32.517128
Value: 111.74424
Policy: 3.8830378
Loss: 15.057462
Value: 474.87125
Policy: -3.2444806
Loss: 44.242645
Value: 90.10698
Policy: -1.3482591
Loss: 7.6624393
Value: 46.15085
Policy: 1.4489247
Loss: 6.0640097
Value: 158.00581
Policy: 0.02140527
Loss: 15.821987
Value: 119.030846
Policy: 0.1362845
Loss: 12.03937
Value: 34.945984
Policy: 0.83226496
Loss: 4.3268633
Value: 425.11557
Policy: 7.3103757
Loss: 49.821934
Value: 86.78198
Policy: 3.2587392
Loss: 11.936938
Value: 152.24913
Policy: 2.0839612
Loss: 17.308874
Value: 446.26077
Policy: -2.1035864
Loss: 42.52249
Value: 483.389
Policy: -3.4307644
Loss: 44.90814
Value: 203.01303
Policy: 3.3203297
Loss: 23.621634
Value: 116.56654
Policy: 3.0341341

...

Value: 470.58035
Policy: -5.958455
Loss: 41.099583
Value: 336.53052
Policy: 1.1710424
Loss: 34.824097
Value: 818.5041
Policy: -12.254506
Loss: 69.5959
Value: 188.62784
Policy: -6.624043
Loss: 12.238741
Value: 307.7397
Policy: 2.4089022
Loss: 33.182873
Value: 256.92368
Policy: -7.563461
Loss: 18.128906
Value: 575.6165
Policy: -0.79268914
Loss: 56.768963
Value: 149.71669
Policy: -3.5250008
Loss: 11.446669
Value: 579.72815
Policy: 5.5156975
Loss: 63.488514
Value: 569.5178
Policy: 6.151234
Loss: 63.103016
Value: 1009.484
Policy: 8.136274
Loss: 109.08468
Value: 689.7577
Policy: 3.685012
Loss: 72.66078
Value: 242.49927
Policy: 3.8272083
Loss: 28.077135
Value: 334.5632
Policy: 4.7473826
Loss: 38.203705
Value: 330.367
Policy: 3.228886
Loss: 36.265587
Value: 186.27489
Policy: 1.0651164
Loss: 19.692606
Value: 285.7425
Policy: 1.4143382
Loss: 29.988588
Value: 474.4343
Policy: -2.40505
Loss: 45.038383
Value: 518.1225
Policy: -0.10090345
Loss: 51.71135
Value: 462.88556
Policy: -1.498826
Loss: 44.78

Value: 508.3275
Policy: -2.7715921
Loss: 48.06116
Value: 482.28897
Policy: 1.0609053
Loss: 49.289803
Value: 242.74184
Policy: -0.42850202
Loss: 23.845682
Value: 433.6753
Policy: 0.6293378
Loss: 43.99687
Value: 494.602
Policy: -2.2285025
Loss: 47.231697
Value: 369.60504
Policy: -3.2572277
Loss: 33.703278
Value: 506.58942
Policy: -3.870935
Loss: 46.78801
Value: 504.3799
Policy: -2.5454428
Loss: 47.892548
Value: 326.54483
Policy: 0.4527986
Loss: 33.10728
Value: 213.21263
Policy: 0.2838441
Loss: 21.605108
Value: 468.20776
Policy: -1.6623245
Loss: 45.158455
Value: 395.84525
Policy: -2.8272014
Loss: 36.757324
Value: 228.2803
Policy: -6.554979
Loss: 16.273052
Value: 215.95767
Policy: 0.4142483
Loss: 22.010015
Value: 281.31152
Policy: 0.14599265
Loss: 28.277145
Value: 384.89114
Policy: -0.90625775
Loss: 37.58286
Value: 376.25186
Policy: 4.6473436
Loss: 42.27253
Value: 203.96545
Policy: 2.2002418
Loss: 22.596786
Value: 370.77377
Policy: -0.7239354
Loss: 36.353443
Value: 224.66164
Policy: 1.4072

Value: 148.19684
Policy: 5.162795
Loss: 19.98248
Value: 59.685905
Policy: 2.4266055
Loss: 8.395196
Value: 120.08989
Policy: -2.5375419
Loss: 9.471447
Value: 105.15069
Policy: 3.786905
Loss: 14.301974
Value: 64.50087
Policy: -0.4148297
Loss: 6.0352573
Value: 94.304855
Policy: 3.7497249
Loss: 13.18021
Value: 92.851524
Policy: 2.4461758
Loss: 11.731328
Value: 88.428825
Policy: -1.1516074
Loss: 7.6912756
Value: 39.929615
Policy: 1.2533377
Loss: 5.2462993
Value: 268.826
Policy: -4.9940357
Loss: 21.888563
Value: 169.11519
Policy: 5.305741
Loss: 22.21726
Value: 734.0266
Policy: -7.450133
Loss: 65.95253
Value: 59.080357
Policy: 2.4146152
Loss: 8.322651
Value: 96.67415
Policy: 0.21037044
Loss: 9.877785
Value: 110.2922
Policy: 3.3261962
Loss: 14.355415
Value: 172.08324
Policy: 3.751563
Loss: 20.959888
Value: 58.363728
Policy: -0.5281954
Loss: 5.3081775
Value: 58.820126
Policy: -1.9439424
Loss: 3.9380703
Value: 785.77014
Policy: -9.136274
Loss: 69.44074
Value: 808.781
Policy: -9.892446
Loss: 70.9

Value: 39.447445
Policy: -0.60491186
Loss: 3.3398328
Value: 262.93863
Policy: 6.3930926
Loss: 32.686954
Value: 41.126556
Policy: 0.93909436
Loss: 5.05175
Value: 43.230785
Policy: -0.26042822
Loss: 4.06265
Value: 777.6338
Policy: 12.828066
Loss: 90.591446
Value: 41.549282
Policy: 1.9853407
Loss: 6.140269
Value: 735.9969
Policy: 12.634367
Loss: 86.23406
Value: 480.98868
Policy: 10.28928
Loss: 58.38815
Value: 727.6288
Policy: 11.725779
Loss: 84.488655
Value: 69.34562
Policy: 2.9678226
Loss: 9.902385
Value: 253.35359
Policy: 6.87596
Loss: 32.21132
Value: 50.83086
Policy: 2.407567
Loss: 7.490653
Value: 638.9566
Policy: 11.946372
Loss: 75.84203
Value: 297.19965
Policy: -0.8800144
Loss: 28.83995
Value: 131.35716
Policy: 2.1365957
Loss: 15.272312
Value: 104.554016
Policy: 4.1706066
Loss: 14.626008
Value: 333.72922
Policy: -1.4861978
Loss: 31.886723
Value: 145.47343
Policy: 3.7313259
Loss: 18.27867
Value: 389.91858
Policy: -3.2989457
Loss: 35.692913
Value: 115.08753
Policy: 3.495179
Loss: 15.00

Value: 807.644
Policy: -7.305144
Loss: 73.45925
Value: 529.22406
Policy: -6.1632857
Loss: 46.75912
Value: 563.64087
Policy: -6.8317165
Loss: 49.53237
Value: 519.8191
Policy: -6.0228057
Loss: 45.959106
Value: 691.67535
Policy: -6.23402
Loss: 62.933514
Value: 257.49802
Policy: -3.663106
Loss: 22.086697
Value: 635.92896
Policy: -7.025389
Loss: 56.567505
Value: 671.79816
Policy: -8.90745
Loss: 58.27237
Value: 170.81223
Policy: 3.8781757
Loss: 20.959398
Value: 49.98745
Policy: 0.5341837
Loss: 5.5329285
Value: 243.01537
Policy: 6.747382
Loss: 31.04892
Value: 142.8672
Policy: 1.9267564
Loss: 16.213476
Value: 194.78024
Policy: 0.30650514
Loss: 19.78453
Value: 165.20581
Policy: 4.847276
Loss: 21.367859
Value: 248.83894
Policy: 5.6115017
Loss: 30.495396
Value: 112.03117
Policy: 1.851073
Loss: 13.054191
Value: 287.80063
Policy: 6.7805905
Loss: 35.560654
Value: 221.7351
Policy: 5.026369
Loss: 27.19988
Value: 177.95908
Policy: -0.79647046
Loss: 16.999437
Value: 263.33563
Policy: -1.0259107
Loss: 25

In [ ]:
action_distribution.get_shape()

In [ ]:
rewards_buf=np.array([[1,2,3,4,5]])
discounted_r = []
v_s_ = 1
for r in rewards_buf[::-1]:
    v_s_ = r + 0.5 * v_s_
    discounted_r.append(v_s_)
discounted_r.reverse()

print(discounted_r)
#self.train(sess, saver, bs, ba, br)

In [ ]:
mb_rewards = [1.0,2.0,3.0]
mb_dones = [False,False,False]
last_value = 1
for n, (rws, dns, vs) in enumerate(zip(mb_rewards, mb_dones, last_value[0])):
    if dns[-1] == 0:
        rws = discount_with_dones(rws+[vs], dns+[0], 0.5)[:-1]
    else:
        rws = discount_with_dones(rws, dns, 0.5)
    #mb_rewards[n] = rws
print(mb_rewards)